#11:45

In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

In [ ]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("Live");
var productContainer = db.GetContainer("Products");

In [ ]:
public class Product    
{
    public string id { get; set; }
    public int ProductId { get; set; }
    public string Name { get; set; }
    public string Desciption { get; set; }
    public int categoryid { get; set; }
    public string CategoryName { get; set; }
    public decimal Price {get;set;}
    public bool InStock {get;set;}
    public List<string> ImageUrls { get; set; }        
}

In [ ]:
var newproduct = new Product
{
    id = Guid.NewGuid().ToString(),
    ProductId = 1,
    Name = "Sample Product",
    Desciption = "This is a sample product.",
    categoryid = 1,
    CategoryName = "Category 1",
    Price = 19.99M,
    InStock = true,
    ImageUrls = new List<string> { "http://example.com/image1.jpg", "http://example.com/image2.jpg" }
};

In [ ]:
var newitem = await productContainer.CreateItemAsync(newproduct, new PartitionKey(newproduct.categoryid));
Console.WriteLine($"Created item in database with id: {newitem.Resource.id}");
Console.WriteLine($"Request charge: {newitem.RequestCharge}");

In [ ]:
var id = 1;
var productQuery = new QueryDefinition("SELECT * FROM c WHERE c.ProductId = @id")
    .WithParameter("@id", id);
var iterator = productContainer.GetItemQueryIterator<Product>(productQuery);
var results = new List<Product>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    Console.WriteLine($"Request charge: {response.RequestCharge}");
}
results[0].Display();

In [ ]:

var item = await productContainer.ReadItemAsync<Product>(results[0].id, new PartitionKey(1));
item.Display();

In [ ]:
item.Resource.Desciption = "Updated";
var updated = await productContainer.ReplaceItemAsync(item.Resource, item.Resource.id, new PartitionKey(1)); 
updated.Display();


In [ ]:
var updated2 = await productContainer.PatchItemAsync<Product>(item.Resource.id, new PartitionKey(1), new[] { PatchOperation.Replace("/Name", "Sample Product Updated") });
updated2.Display()

In [ ]:
var temp = await productContainer.DeleteItemAsync<Product>(item.Resource.id, new PartitionKey(1));
temp.Display();

#12:00